## Libraries

In [40]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from reportlab.lib.pagesizes import landscape, letter
from reportlab.pdfgen import canvas
from PIL import Image
import tempfile
import os

## Getting Data

In [41]:
df = pd.read_csv('adult.data.csv')

In [42]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [43]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

In [44]:
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


Checking for missing values

In [45]:
def func_percent_values_missing(df: pd.DataFrame):
    if not isinstance(df, pd.DataFrame):
        raise ValueError("O argumento fornecido não é um DataFrame do Pandas")
    
    totalCells = np.product(df.shape)
    missingCount = df.isnull().sum()
    totalMissing = missingCount.sum()
    print("O dataset tem", round(((totalMissing/totalCells) * 100), 2), "% de valores ausentes")

func_percent_values_missing(df)

O dataset tem 0.0 % de valores ausentes


C:\Users\ana.alves\AppData\Local\Temp\ipykernel_19484\2855485466.py:10: DeprecationWarning:

`product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.



Since df has no missing values, it's not necessary check or due another analysis for missing values.

## Data Analysis

In [46]:
racas_contagem = df['race'].value_counts().reset_index()
racas_contagem.columns = ['Race', 'Count']

quantidades = racas_contagem['Count'].tolist()


racas = racas_contagem['Race'].tolist()

color_list = ['#8fbee0', '#9ccbed', '#83d3dc', '#83d6c6', '#98d5af', '#a9cab4', '#c5b2be', '#d1a5c3', '#c59ab7', '#b98fac', '#ae84a1', '#a29fbd', '#9895b2', 
    '#8d8ba8', '#9694b1', '#9f9dbb', '#a8a6c4']

fig1 = go.Figure()

fig1.add_trace(go.Pie(
    labels=racas,
    values=quantidades,
    hole=0.4,  
    marker=dict(colors=color_list),  
))

fig1.update_layout(
    title="Quantidade de pessoas de cada raça",
)

fig1.write_image("fig1.png", format="png")

In [47]:
idade_media_homem = df[df['sex'] == 'Male']['age'].mean()
idade_media_mulher = df[df['sex'] == 'Female']['age'].mean()

fig2 = go.Figure(data=[
    go.Bar(name='Homens', x=['Média de idade homens'], y=[idade_media_homem], marker_color='#8fbee0'),
    go.Bar(name='Mulheres', x=['Média de idade mulheres'], y=[idade_media_mulher], marker_color='#d1a5c3')
])

fig2.update_layout(
    title="Comparação da idade média entre homens e mulheres",
    yaxis_title="Idade Média",
    barmode='group', 
)

fig2.write_image("fig2.png", format="png")

In [48]:
porcentagem_bacharelado = (df['education'] == "Bachelors").mean() * 100

fig3 = go.Figure(data=[go.Pie(
    labels=['Com Bacharelado', 'Sem Bacharelado'],
    values=[porcentagem_bacharelado, 100 - porcentagem_bacharelado],
    hole=0.4, 
    marker=dict(colors=color_list[:2]) 
)])

fig3.update_layout(
    title="Porcentagem de pessoas com Bacharelado",
)

fig3.write_image("fig3.png", format="png")


In [49]:
porcentagem_educacao_avancada_mais50k = ((df['education'].isin(['Bachelors', 'Masters', 'Doctorate'])) & (df['salary'] == '>50K')).mean() * 100

fig4 = go.Figure(data=[go.Pie(
    labels=['Pós graduação e renda >50K', 'Pós graduação e renda <=50K'],
    values=[porcentagem_educacao_avancada_mais50k, 100 - porcentagem_educacao_avancada_mais50k],
    hole=0.4,  
    marker=dict(colors=color_list[:2])  
)])

fig4.update_layout(
    title="Porcentagem de pessoas com pós graduação e com renda maior que 50 mil",
)

fig4.write_image("fig4.png", format="png")


In [50]:
min_horas_semana = df['hours-per-week'].min()
max_horas_semana = df['hours-per-week'].max()

fig5 = go.Figure(data=[go.Bar(
    x=['Mínimo', 'Máximo'],
    y=[min_horas_semana, max_horas_semana],
    marker_color=color_list[:2] 
)])

fig5.update_layout(
    title="Número mínimo e máximo de horas trabalhadas por semana",
    xaxis_title="Tipo de Horas",
    yaxis_title="Número de Horas",
)

fig5.write_image("fig5.png", format="png")



In [51]:
porcentagem_min_horas_mais50k = ((df['hours-per-week'] == min_horas_semana) & (df['salary'] == '>50K')).mean() * 100
porcentagem_max_horas_mais50k = ((df['hours-per-week'] == max_horas_semana) & (df['salary'] == '>50K')).mean() * 100

fig6 = go.Figure(data=[go.Bar(
    x=['Mínimo', 'Máximo'],
    y=[porcentagem_min_horas_mais50k, porcentagem_max_horas_mais50k],
    marker_color=color_list[:2] 
)])

fig6.update_layout(
    title="Porcentagem de pessoas que ganham mais de 50K",
    xaxis_title="Horas de Trabalho",
    yaxis_title="Porcentagem",
)

fig6.write_image("fig6.png", format="png")



In [52]:

porcentagem_pais_mais50k = df[df['salary'] == '>50K']['native-country'].value_counts(normalize=True) * 100
pais_maior_porcentagem = porcentagem_pais_mais50k.idxmax()
maior_porcentagem = porcentagem_pais_mais50k.max()

fig7 = go.Figure(data=[go.Bar(
    x=porcentagem_pais_mais50k.index,
    y=porcentagem_pais_mais50k,
    marker_color=color_list[:len(porcentagem_pais_mais50k)]  # Especificando as cores
)])
fig7.update_layout(
    title="Porcentagem de pessoas que ganham mais de 50K por país",
    xaxis_title="País",
    yaxis_title="Porcentagem",
)

fig7.write_image("fig7.png", format="png")
print(f"País com a maior porcentagem de pessoas que ganham mais de 50K: {pais_maior_porcentagem}")
print(f"Porcentagem correspondente: {maior_porcentagem}%")



País com a maior porcentagem de pessoas que ganham mais de 50K: United-States
Porcentagem correspondente: 91.45517153424308%


In [53]:
df_india_mais50k = df[(df['native-country'] == 'India') & (df['salary'] == '>50K')]
contagem_ocupacao = df_india_mais50k['occupation'].value_counts()

fig8 = go.Figure(data=[go.Bar(
    x=contagem_ocupacao.index,
    y=contagem_ocupacao,
    marker_color=color_list[:len(contagem_ocupacao)] 
)])


fig8.update_layout(
    title="Ocupações mais populares para quem ganha mais de 50K na Índia",
    xaxis_title="Ocupação",
    yaxis_title="Contagem",
)

fig8.write_image("fig8.png", format="png")


## Gerar PDF


In [54]:

figuras = [fig1, fig2, fig3, fig4, fig5, fig6, fig7]

# Crie um diretório temporário para armazenar os arquivos de imagem
with tempfile.TemporaryDirectory() as tmpdirname:
    try:
        for i, fig in enumerate(figuras):
            # Renderize o gráfico como uma imagem e salve-a no diretório temporário
            img_file = os.path.join(tmpdirname, f"fig_{i}.png")
            fig.write_image(img_file, format="png")

        # Crie um arquivo PDF
        pdf_file = "graficos.pdf"
        c = canvas.Canvas(pdf_file, pagesize=landscape(letter))

        # Adicione cada imagem PNG ao PDF
        for i in range(len(figuras)):
            img_file = os.path.join(tmpdirname, f"fig_{i}.png")
            img = Image.open(img_file)
            c.drawInlineImage(img_file, 0, 0, width=landscape(letter)[0], height=landscape(letter)[1])
            c.showPage()

        # Salve o PDF
        c.save()

        print(f"Arquivo PDF '{pdf_file}' gerado com sucesso!")
        
    except Exception as e:
        print(f"Erro ao gerar PDF: {e}")


Arquivo PDF 'graficos.pdf' gerado com sucesso!


NotADirectoryError: [WinError 267] O nome do diretório é inválido: 'C:\\Users\\ANA~1.ALV\\AppData\\Local\\Temp\\tmp7_rldeoe\\fig_6.png'